In [1]:
from common import *
from gensim.models.wrappers import FastText

import io

In [15]:
sw = set(extra_stop_words)

with io.open('../data/corpus_raw.txt', 'r', encoding='utf-8') as fr:
    with io.open('../data/_corpus_raw.txt', 'w', encoding='utf-8') as fw:
        for line in fr:
            sents = sent_tokenize(line.lower())
            sents = [[w for w in word_tokenize(s) if w not in sw] for s in sents]
            if len(new_line):
                fw.write('. '.join(' '.join(s) for s in sents) + u'\n')

KeyboardInterrupt: 

In [4]:
line = u'1. опорный подшипник скольжения , содержащий корпус , вкладыш с помехами. Задача решена'

sents = sent_tokenize(line.lower())
sents = [[w for w in word_tokenize(s) ] for s in sents]

print('. '.join(' '.join(s) for s in sents))


1. опорный подшипник скольжения , содержащий корпус , вкладыш с помехами .. задача решена


In [1]:
# model = FastText.train('/root/fastText/fasttext', '../data/_corpus_raw.txt', 
#                        size=100, 
#                        model='skipgram',
#                        window=5, 
#                        min_count=5, 
#                        iter=5,
#                        threads=cpu_count)

In [5]:
model = gensim.models.KeyedVectors.load_word2vec_format('../data/ft_output.vec')

2017-10-03 11:31:36,971 [MainThread  ] [INFO ]  loading projection weights from ../data/ft_output.vec
2017-10-03 11:42:32,561 [MainThread  ] [INFO ]  loaded (3648917, 100) matrix from ../data/ft_output.vec


In [6]:
most = model.most_similar(u'стол')
for w, score in most:
    print(w + ', %s' % score)

2017-10-03 11:42:32,580 [MainThread  ] [INFO ]  precomputing L2-norms of word weight vectors


стол,, 0.882844150066
столик, 0.872067809105
стол-рольганг, 0.834802746773
столом, 0.83262771368
стол-штатив, 0.828676521778
стола, 0.813986182213
стол-лоток, 0.813705742359
стол., 0.810664772987
стол-стеллаж, 0.808469176292
стол), 0.803629040718


In [7]:
with open(join(DATA_FOLDER, 'gold_mongo.json'), 'r') as f:
    gold = json.load(f)

In [ ]:
folder = join(DATA_FOLDER, 'documents_raw/')
fnames = glob(join(folder, '*.gz'))

def worker(fnames, phraser):
    vecs = []
    keys = []
    for k, doc in tqdm_notebook(iter_docs(fnames)):
        _doc = phraser[doc]
        word_vecs = np.array([wv[w] for s in _doc for w in s if w in wv], dtype=np.float32)
        if len(word_vecs):
            doc_vec = word_vecs.mean(axis=0)
            vecs.append(doc_vec)
            keys.append(k)
    return vecs, keys

In [ ]:
import gc

vecs = []
for _names in tqdm_notebook(np.array_split(fnames, 80)):
    parallelizer = Parallel(n_jobs=cpu_count)

    # this iterator returns the functions to execute for each task
    tasks_iterator = ( delayed(worker)(list_block, lambda x: x) for 
                      list_block in grouper(1, _names) ) 
    result = parallelizer( tasks_iterator )
    vecs.append(pd.concat(result))
    gc.collect()

In [ ]:
conc = pd.concat(vecs)
np.save('../data/saved/vecs_bigram.npy', conc.values)
np.save('../data/saved/ids_bigram.npy', conc.index)

In [10]:
def predict(keys, lim=200):
    names = pd.Series(range(len(fnames)), index = fnames)
    test_vecs = []
    for k in keys:
        ix = names[k]
        if isinstance(ix, pd.Series):
            ix = ix[0]
        tvec = doc_embeds_reshaped[ix]
        test_vecs.append(tvec)

    dists = euclidean_distances(test_vecs, doc_embeds_reshaped)
    sorted_ixs = np.argsort(dists, axis=1)
    preds = {}
    for k, _ixs in zip(keys, sorted_ixs):
        preds[k] = [n for n in fnames[_ixs[1:lim+1]]]
        
    return preds

In [11]:
preds = predict(gold.keys(), lim=200)

NameError: global name 'fnames' is not defined

In [ ]:
result = evaluate(preds, gold)

ax = result['acc10'].hist()
ax.set_xlabel("acc10")
plt.show()

ax = result['acc20'].hist()
ax.set_xlabel("acc20")
plt.show()

ax = result['acc200'].hist()
ax.set_xlabel("acc200")
plt.show()